In [ ]:
# multivariate multi-step encoder-decoder lstm example
import numpy as np
from numpy import array
from numpy import hstack
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd
import csv
from math import ceil

path = '/content/drive/MyDrive/train.csv'
data_frame = pd.read_csv(path)
data1 = data_frame#.drop(columns=['MST','DATE (MM/DD)','Global CMP22 (vent/cor) [W/m^2]',
#                                'Direct sNIP [W/m^2]', 'Azimuth Angle [degrees]',
#                                'Precipitation (Accumulated) [mm]','Snow Depth [cm]', 'Moisture', 'Albedo (CMP11)'])
data = data1.loc[data1['Total Cloud Cover [%]']>-1]
data = data.loc[data['Total Cloud Cover [%]']<101]

In [ ]:
# define input sequence
#data = pd.read_csv("/content/drive/MyDrive/train.csv")
in_seqa = array(data['Tower Dry Bulb Temp [deg C]'].values)  # as a numpy array
in_seqb = array(data['Tower Wet Bulb Temp [deg C]'].values)  # as a numpy array
in_seqc = array(data['Tower Dew Point Temp [deg C]'].values)  # as a numpy array
in_seqd = array(data['Tower RH [%]'].values)  # as a numpy array
in_seqe = array(data['Peak Wind Speed @ 6ft [m/s]'].values)  # as a numpy array
in_seqf = array(data['Avg Wind Direction @ 6ft [deg from N]'].values)  # as a numpy array
in_seqg = array(data['Station Pressure [mBar]'].values)  # as a numpy array
in_seqh = array(data['Precipitation (Accumulated) [mm]'].values)  # as a numpy array
in_seqi = array(data['Moisture'].values)  # as a numpy array
in_seqj = array(data['Direct sNIP [W/m^2]'].values)  # as a numpy array

out_seqa = array(data['Total Cloud Cover [%]'].values)  # as a numpy array

# convert to [rows, columns] structure
in_seqa = in_seqa.reshape((len(in_seqa), 1))
in_seqb = in_seqb.reshape((len(in_seqb), 1))
in_seqc = in_seqc.reshape((len(in_seqc), 1))
in_seqd = in_seqd.reshape((len(in_seqd), 1))
in_seqe = in_seqe.reshape((len(in_seqe), 1))
in_seqf = in_seqf.reshape((len(in_seqf), 1))
in_seqg = in_seqg.reshape((len(in_seqg), 1))
in_seqh = in_seqh.reshape((len(in_seqh), 1))
in_seqi = in_seqi.reshape((len(in_seqi), 1))
in_seqj = in_seqj.reshape((len(in_seqj), 1))

out_seqa = out_seqa.reshape((len(out_seqa), 1))

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out
    # check if we are beyond the dataset
    if out_end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
# horizontally stack columns
dataset = hstack((out_seqa, in_seqi))#, in_seqh))#in_seqa, in_seqb, in_seqc, in_seqd, out_seqa, in_seqe, in_seqf, in_seqg))
# choose a number of time steps
n_steps_in, n_steps_out = 1, 1
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [ ]:
# define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(20, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='Adam', loss='mse', metrics=['acc'])
#from keras import backend as K
#K.set_value(model.optimizer.learning_rate, 0.0001)

In [ ]:
#fit model
model.fit(X, y, epochs=100, verbose=1)#batch_size = 64, epochs=10, verbose=1)

Epoch 1/100
7836/7836 [==============================] - 26s 3ms/step - loss: 22.1002 - acc: 0.9987
Epoch 2/100
7836/7836 [==============================] - 23s 3ms/step - loss: 3.0455 - acc: 1.0000
Epoch 3/100
7836/7836 [==============================] - 23s 3ms/step - loss: 3.0455 - acc: 1.0000
Epoch 4/100
7836/7836 [==============================] - 23s 3ms/step - loss: 3.0430 - acc: 1.0000
Epoch 5/100
7836/7836 [==============================] - 23s 3ms/step - loss: 3.0428 - acc: 1.0000
Epoch 6/100
7836/7836 [==============================] - 23s 3ms/step - loss: 3.0393 - acc: 1.0000
Epoch 7/100
7836/7836 [==============================] - 26s 3ms/step - loss: 3.0344 - acc: 1.0000
Epoch 8/100
7836/7836 [==============================] - 31s 4ms/step - loss: 3.0415 - acc: 1.0000
Epoch 9/100
7836/7836 [==============================] - 27s 3ms/step - loss: 3.0353 - acc: 1.0000
Epoch 10/100
7836/7836 [==============================] - 22s 3ms/step - loss: 3.0422 - acc: 1.0000
Epoch 11

In [ ]:
# input = X[0]
# input = input.reshape((input.shape[0], n_steps_in, n_features))
# output = model.predict(input, verbose=0)
# print(output)

In [ ]:
# import matplotlib.pyplot as plt
# plt.scatter(data['Direct sNIP [W/m^2]'], data['Total Cloud Cover [%]'])
# plt.show()

In [ ]:
#model.save('/content/drive/MyDrive/89_10167')
#model = keras.models.load_model('/content/drive/MyDrive/Model/')

# from sklearn.externals import joblib
# Save the model as a pickle in a file
# joblib.dump(model, '/content/drive/MyDrive/89_10167')
# Load the model from the file
# model = joblib.load('/content/drive/MyDrive/89_10167')
# Use the loaded model to make predictions
#model.predict(X_test)

In [ ]:
#test_data input
List = []
List__30 = []
List__60 = []
List__90 = []
List__120 = []
A = 0
N = 301
k = 0
for i in range(1,N):
  j= str(i)
  k = k + 1
  path_ = '/content/drive/MyDrive/Hack/Shell.ai Hackathon/dataset/test/' + j + '/weather_data.csv'
  data_frame_ = pd.read_csv(path_)
  data_ = data_frame_.drop(columns=['Global CMP22 (vent/cor) [W/m^2]',
                                  'Direct sNIP [W/m^2]', 'Azimuth Angle [degrees]',
                                  'Snow Depth [cm]', 'Albedo (CMP11)', 'Tower Dry Bulb Temp [deg C]', 
                                  'Tower Wet Bulb Temp [deg C]', 'Tower RH [%]', 'Tower Dew Point Temp [deg C]', 
                                  'Peak Wind Speed @ 6ft [m/s]', 
                                  'Avg Wind Direction @ 6ft [deg from N]', 'Station Pressure [mBar]'
                                  ,'Precipitation (Accumulated) [mm]'])#, 'Moisture'])
  data1_ = data_.loc[data_['Total Cloud Cover [%]']>-1]
  data1_ = data1_.loc[data_['Total Cloud Cover [%]']<101]
  X_test = data1_.drop('Time [Mins]', axis = 1)
  X_test = array(X_test.values)
  X_ = X_test[X_test.shape[0] - 1]
  x_input = X_
  x_input = x_input.reshape(1, X_.shape[0])
  # demonstrate prediction
  x_input = x_input.reshape((x_input.shape[0], n_steps_in, n_features))
  for i in range(120):
    yhat = model.predict(x_input, verbose=0)
    if i == 29:
      pred_30 = yhat[0][0][A]
      if pred_30 > 100:
        pred_30 = 100
      elif pred_30 < 0:
        pred_30 = 0
      List__30.append(pred_30)
#      print(pred_30)#ceil(yhat[0][0][4]))
    elif i == 59:
      pred_60 = yhat[0][0][A]
      if pred_60 > 100:
        pred_60 = 100
      elif pred_60 < 0:
        pred_60 = 0
      List__60.append(pred_60)
#      print(pred_60)#ceil(yhat[0][0][4]))
    elif i == 89:
      pred_90 = yhat[0][0][A]
      if pred_90 > 100:
        pred_90 = 100
      elif pred_90 < 0:
        pred_90 = 0
      List__90.append(pred_90)
#      print(pred_90)#ceil(yhat[0][0][4]))
    elif i == 119:
      pred_120 = yhat[0][0][A]
      if pred_120 > 100:
        pred_120 = 100
      elif pred_120 < 0:
        pred_120 = 0
      List__120.append(pred_120)
#      print(pred_120)#ceil(yhat[0][0][7]))
      print(k)
    x_input = yhat[0][0]
    x_input = x_input.reshape((1 , n_features))
    x_input = x_input.reshape((x_input.shape[0], n_steps_in, n_features))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
#writing to file
Details = ['scenario_set','30_min_horizon', '60_min_horizon', '90_min_horizon', '120_min_horizon']  
fin = []
for i in range(0,N-1):
  fin.append((i+1, List__30[i], List__60[i], List__90[i], List__120[i]))
with open('/content/drive/MyDrive/finalfloat.csv', "w") as f:
  writer = csv.writer(f, lineterminator='\n')
  writer.writerow(['scenario_set', '30_min_horizon', '60_min_horizon', '90_min_horizon', '120_min_horizon'])
  writer.writerows(fin)

In [ ]:
for i in range(0, N-1):
  List__30[i] = np.ceil(int(List__30[i]))
  List__60[i] = np.ceil(int(List__60[i]))
  List__90[i] = np.ceil(int(List__90[i]))
  List__120[i] = np.ceil(int(List__120[i]))

In [ ]:
#writing to file
Details = ['scenario_set','30_min_horizon', '60_min_horizon', '90_min_horizon', '120_min_horizon']  
fin = []
for i in range(0,N-1):
  fin.append((i+1, List__30[i], List__60[i], List__90[i], List__120[i]))
with open('/content/drive/MyDrive/finalceil.csv', "w") as f:
  writer = csv.writer(f, lineterminator='\n')
  writer.writerow(['scenario_set', '30_min_horizon', '60_min_horizon', '90_min_horizon', '120_min_horizon'])
  writer.writerows(fin)

In [ ]:
for i in range(0, N-1):
 List__30[i] = round(int(List__30[i]))
 List__60[i] = round(int(List__60[i]))
 List__90[i] = round(int(List__90[i]))
 List__120[i] = round(int(List__120[i]))

In [ ]:
path = '/content/drive/MyDrive/final.csv'
df = pd.read_csv(path,  index_col=False)
k = 0.98
l = 100
l1 = 10
l2 = 7
df['30_min_horizon'] = np.ceil(df['30_min_horizon']* k)
df['60_min_horizon'] = np.ceil(df['60_min_horizon']* k)
df['90_min_horizon'] = np.ceil(df['90_min_horizon']* k)
df['120_min_horizon'] = np.ceil(df['120_min_horizon']* k)
df.to_csv('/content/drive/MyDrive/finalceil.csv.csv', index = False)
csv_file = pd.read_csv('/content/drive/MyDrive/finalceil.csv.csv')
dict_obj = csv_file.to_dict('records')
for row in dict_obj:
    row['30_min_horizon'] = 100 if row['30_min_horizon'] > l else row['30_min_horizon']
    row['60_min_horizon'] = 100 if row['60_min_horizon'] > l else row['60_min_horizon']
    row['90_min_horizon'] = 100 if row['90_min_horizon'] > l else row['90_min_horizon']
    row['120_min_horizon'] = 100 if row['120_min_horizon'] > l else row['120_min_horizon']
    row['30_min_horizon'] = l2 if row['30_min_horizon'] < l1 else row['30_min_horizon']
    row['60_min_horizon'] = l2 if row['60_min_horizon'] < l1 else row['60_min_horizon']
    row['90_min_horizon'] = l2 if row['90_min_horizon'] < l1 else row['90_min_horizon']
    row['120_min_horizon'] = l2 if row['120_min_horizon'] < l1 else row['120_min_horizon']
df1 = pd.DataFrame(dict_obj)
df1.to_csv('/content/drive/MyDrive/finalceil.csv.csv', index = False) 